#OCR for school profile

##Download data

In [1]:
import os
import shutil

gd_label_dir = '/kaggle/input/school-profile/HB/Labeled'
label_files = os.listdir(gd_label_dir)

In [2]:
gd_img_dir = '/kaggle/input/school-profile/HB/Original'
dataset_folder = '/kaggle/working/datasets/custom_data'
os.makedirs(dataset_folder, exist_ok=True)

img_folders = os.listdir(gd_img_dir)
#os.makedirs(img_dir, exist_ok=True)
#print(*label_files, sep='\n')

file_names = set([label_file.split('/')[-1][:-7] for label_file in label_files])
for name in file_names:
    if name in img_folders:
        imgs = os.listdir(os.path.join(gd_img_dir, name))
        #print(imgs)
        for img in imgs:
            if os.path.exists(os.path.join(gd_label_dir, img[:-3]+'txt')):
                shutil.copy(os.path.join(gd_img_dir, name, img), dataset_folder)
                shutil.copy(os.path.join(gd_label_dir, img[:-3]+'txt'), dataset_folder)

In [3]:
import random

valid_test_size = 0.3

valid_samples = set(random.sample(img_folders, int(valid_test_size*len(img_folders))))
train_samples = set(img_folders) - valid_samples
test_samples = set(random.sample(valid_samples, int(0.25*len(valid_samples))))
valid_samples = valid_samples - test_samples

print("test_samples:", len(test_samples))
print("train_samples:", len(train_samples))
print("valid_samples:", len(valid_samples))

with open("/kaggle/working/datasets/custom_train.txt", 'w') as f:
    for sample in train_samples:
        imgs = os.listdir(os.path.join(gd_img_dir, sample))
        for img in imgs:
            f.write('/kaggle/working/datasets/custom_data/'+img+'\n')

with open("/kaggle/working/datasets/custom_valid.txt", 'w') as f:
    for sample in valid_samples:
        imgs = os.listdir(os.path.join(gd_img_dir, sample))
        for img in imgs:
            f.write('/kaggle/working/datasets/custom_data/'+img+'\n')

with open("/kaggle/working/datasets/custom_test.txt", 'w') as f:
    for sample in test_samples:
        imgs = os.listdir(os.path.join(gd_img_dir, sample))
        for img in imgs:
            f.write('/kaggle/working/datasets/custom_data/'+img+'\n')

test_samples: 4
train_samples: 39
valid_samples: 12


/tmp/ipykernel_25/3456140663.py:7: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_samples = set(random.sample(valid_samples, int(0.25*len(valid_samples))))


In [4]:
classes_file = '/kaggle/input/school-profile/HB/Labeled/classes.txt'
with open(classes_file, 'r') as f:
    classes = f.read().strip().split('\n')
custom_content = f"path: .\n\
train: custom_train.txt\n\
val:  custom_valid.txt\n\
test:  custom_train_test.txt\n\
nc: {len(classes)}  # number of classes\n\
names: {classes}  # class names\n\
"

with open('/kaggle/working/custom.yaml', 'w') as f:
    f.write(custom_content)

##Yolov8

In [5]:
!pip install --upgrade ultralytics==8.0.186 wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.4/618.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 47.0 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.16.6
    Uninstalling wandb-0.16.6:
      Successfully uninstalled wandb-0.16.6


In [6]:
import wandb
wandb.login(key='e9f98fa97698b1759125dc076e2195f407c2d4af')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



In [8]:
from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback


# Step 1: Initialize a Weights & Biases run
wandb.init(project="ocr_yolov8", job_type="training")

# Step 2: Define the YOLOv8 Model and Dataset
model_name = "yolov8s"
dataset_name = "custom.yaml"
model = YOLO(f"{model_name}.pt")

# Step 3: Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

# Step 4: Train and Fine-Tune the Model
model.train(project="ocr_yolov8", data=dataset_name, epochs=150, imgsz=640, batch=32, save_period=-1)

# Step 5: Validate the Model
model.val()

# Step 6: Perform Inference and Log Results
#model(["path/to/image1", "path/to/image2"])

# Step 7: Finalize the W&B Run
wandb.finish()

wandb: Currently logged in as: thanh_nd (thanh_nd_ai). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240530_032407-rx9sxf7w
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run atomic-hill-10
wandb: ⭐️ View project at https://wandb.ai/thanh_nd_ai/ocr_yolov8
wandb: 🚀 View run at https://wandb.ai/thanh_nd_ai/ocr_yolov8/runs/rx9sxf7w
100%|██████████| 21.5M/21.5M [00:00<00:00, 165MB/s]
New https://pypi.org/project/ultralytics/8.2.25 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.186 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=custom.yaml, epochs=150, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=ocr_yolov8, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, 

Generating Visualizations for batch-1/1:   0%|          | 0/64 [00:00<?, ?it/s]


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      2/150      7.92G      2.023       4.61        1.6        369        640: 100%|██████████| 10/10 [00:05<00:00,  1.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.33s/it]
                   all         96        841      0.803     0.0818      0.108     0.0818


Generating Visualizations for batch-1/1:   0%|          | 0/32 [00:00<?, ?it/s]


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      3/150      8.03G      1.466      2.857      1.279        297        640: 100%|██████████| 10/10 [00:04<00:00,  2.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.29s/it]
                   all         96        841      0.472      0.243      0.202      0.153


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    140/150       8.1G      0.567     0.3718     0.8709        309        640: 100%|██████████| 10/10 [00:04<00:00,  2.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.97it/s]
                   all         96        841      0.938      0.967      0.965      0.714
Stopping training early as no improvement observed in last 50 epochs. Best results observed at epoch 90, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.


Generating Visualizations for batch-1/1:   0%|          | 0/32 [00:00<?, ?it/s]


140 epochs completed in 1.797 hours.
Optimizer stripped from ocr_yolov8/train/weights/last.pt, 22.5MB
Optimizer stripped from ocr_yolov8/train/weights/best.pt, 22.5MB

Validating ocr_yolov8/train/weights/best.pt...
Ultralytics YOLOv8.0.186 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11136807 parameters, 0 gradients
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.53it/s]
                   all         96        841      0.954      0.947       0.97       0.72
                serial         96         12          1      0.965      0.995      0.713
                  name         96         70      0.978      0.986      0.989      0.779
                gender         96         11      0.907          1      0.972      0.664
                   dob         96         12      0.881      0.917      0.968      0.675
            birthplace         96         12      0.9

Generating Visualizations for batch-1/1:   0%|          | 0/32 [00:00<?, ?it/s]

Speed: 0.2ms preprocess, 7.2ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to ocr_yolov8/val
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▂▃▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:                  lr/pg1 ▂▃▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:                  lr/pg2 ▂▃▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:        metrics/mAP50(B) ▁▃▆▇████████████████████████████████████
wandb:     metrics/mAP50-95(B) ▁▃▅▆▇▇▇▇▇▇█▇████████████████████████████
wandb:    metrics/precision(B) ▅▁▅▆▆▆▆▇▇▇█▇▇▇▇██▇▇▇██▇▇████████████████
wandb:       metrics/recall(B) ▁▃▅▆▇▇▇███▇█████████████████████████████
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          train/cls_loss █▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          train/df